# Clustering News
### This dataset has been downloaded from https://www.kaggle.com/rmisra/news-category-dataset.
### Our goal is to categorize news articles based on their headlines and short descriptions

In [1]:
# Import of libs

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
#Naive Bayes
from sklearn.naive_bayes import MultinomialNB
# K-Means
from sklearn.cluster import KMeans
#confusion Matrix
from sklearn.metrics import confusion_matrix
# Score
from sklearn.metrics import classification_report

In [2]:
#Load the dataset
news = pd.read_json('/media/sf_FormacaoCientistaDeDados/Portfolio/News_Category/news-category-dataset/News_Category_Dataset_v2.json'
                    ,lines=True)

In [3]:
news.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [5]:
#Checking missing values, columns and rowcount
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200853 entries, 0 to 200852
Data columns (total 6 columns):
authors              200853 non-null object
category             200853 non-null object
date                 200853 non-null datetime64[ns]
headline             200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 10.7+ MB


In [5]:
news['text']= news['headline'] + ' '+news['short_description']

In [7]:
news.groupby(['category']).count()

,authors,date,headline,link,short_description,text
category,,,,,,
ARTS,1509,1509,1509,1509,1509,1509
ARTS & CULTURE,1339,1339,1339,1339,1339,1339
BLACK VOICES,4528,4528,4528,4528,4528,4528
BUSINESS,5937,5937,5937,5937,5937,5937
COLLEGE,1144,1144,1144,1144,1144,1144
COMEDY,5175,5175,5175,5175,5175,5175
CRIME,3405,3405,3405,3405,3405,3405
CULTURE & ARTS,1030,1030,1030,1030,1030,1030
DIVORCE,3426,3426,3426,3426,3426,3426


In [46]:
#Selecting some categories
news_2 = news.loc[news['category'].isin(['RELIGION','SCIENCE','MEDIA','WEIRD NEWS','STYLE','TASTE','TECH'])]


In [47]:
# rowcount
news_2.shape

(16651, 7)

In [4]:
msk = np.random.rand(len(news)) < 0.05
news_2 = news[msk]
    

In [5]:
#count of "category"
len(news_2['category'].unique())

41

In [7]:
#Grouping categories into Topics
news_2.loc[news_2['category']=='POLITICS','topic']='POLITICS'
news_2.loc[news_2['category']=='ENTERTAINMENT','topic']='ENTERTAINMENT'



In [ ]:
news_2.loc[(news_2['category']!='ENTERTAINMENT')& (news_2['category']!='POLITICS'),'topic']='OTHERS'

## Preprocessing
#### We'll to turn "text" feature into word vectors

In [48]:
# Function for preprocessing
def preprocessing(text):
    #removing the punctuation
    text_v2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in text]).split())
    #tokenizes sentences into words
    tokens = [word for sent in nltk.sent_tokenize(text_v2) for word in nltk.word_tokenize(sent)]
    #lowercase
    tokens =[word.lower() for word in tokens]
    #Removing stopwords
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    #Removing words lenght <  3
    tokens = [token for token in tokens if len(tokens) >=3 ]
    #Stemming ("eating" into "eat")
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    #POS tagging
    tagged_corpus = pos_tag(tokens)
    
    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']
    lemmatizer = WordNetLemmatizer()
    
    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n')
        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v')
        else:
            return lemmatizer.lemmatize(token,'n')
    
    
    pre_proc_text = ' '.join([prat_lemmatize(token,tag)for token ,tag in tagged_corpus])
    
    return pre_proc_text
    
    

In [49]:
# Applies the preprocessing function
news_3 = []
for r in news_2['text']:
    news_3.append(preprocessing(r))
    

In [50]:
#split train and test dataset
x_train, x_test, y_train, y_test = train_test_split(news_3, news_2['category'])

In [51]:
#building TFIDF Vectorizer
vectorizer = TfidfVectorizer(min_df=3 ,stop_words ='english',strip_accents='unicode',ngram_range=(1,3))#,norm='l1')
x_train_2 = vectorizer.fit_transform(x_train).todense()

## Naive Bayes

In [52]:
# Training the model
nb = MultinomialNB().fit(x_train_2,y_train)
#Predict
nb_predicted = nb.predict(x_train_2)


In [53]:
# Evalueate test dataset 
x_test_2 = vectorizer.transform(x_test).todense() 
nb_predicted_test = nb.predict(x_test_2)

In [58]:
#confusion_matrix 
pd.crosstab(y_test, nb_predicted_test,rownames =["Actuall"],colnames = ["Predicted"])

Predicted,MEDIA,RELIGION,SCIENCE,STYLE,TASTE,TECH,WEIRD NEWS
Actuall,,,,,,,
MEDIA,623,28,5,7,1,19,25
RELIGION,44,561,10,10,5,7,22
SCIENCE,21,25,427,12,10,14,44
STYLE,36,13,9,435,18,8,39
TASTE,18,8,6,19,416,13,31
TECH,52,13,17,13,7,375,42
WEIRD NEWS,33,14,28,28,41,9,502


In [59]:
# score on test dataset
print(classification_report(y_test, nb_predicted_test))

             precision    recall  f1-score   support

      MEDIA       0.75      0.88      0.81       708
   RELIGION       0.85      0.85      0.85       659
    SCIENCE       0.85      0.77      0.81       553
      STYLE       0.83      0.78      0.80       558
      TASTE       0.84      0.81      0.82       511
       TECH       0.84      0.72      0.78       519
 WEIRD NEWS       0.71      0.77      0.74       655

avg / total       0.81      0.80      0.80      4163



## K-Means

In [40]:
# K-means
km = KMeans(n_clusters = 4,init= 'random',n_init=1,verbose = 1)
km.fit(x_train_2)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 7154.9458169
start iteration
done sorting
end inner loop
Iteration 1, inertia 7141.32938551
start iteration
done sorting
end inner loop
Iteration 2, inertia 7135.23071076
start iteration
done sorting
end inner loop
Iteration 3, inertia 7132.60706369
start iteration
done sorting
end inner loop
Iteration 4, inertia 7130.91423168
start iteration
done sorting
end inner loop
Iteration 5, inertia 7129.44015941
start iteration
done sorting
end inner loop
Iteration 6, inertia 7127.82931463
start iteration
done sorting
end inner loop
Iteration 7, inertia 7124.80250394
start iteration
done sorting
end inner loop
Iteration 8, inertia 7118.27333936
start iteration
done sorting
end inner loop
Iteration 9, inertia 7104.18822254
start iteration
done sorting
end inner loop
Iteration 10, inertia 7083.1160202
start iteration
done sorting
end inner loop
Iteration 11, inertia 7082.93659652
start iteration
done sorting

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300,
    n_clusters=4, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)

In [41]:
km.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [43]:
km.labels_.shape

(7243,)

In [44]:
km.cluster_centers_

array([[  1.10199847e-04,   2.55529676e-03,   1.67927139e-04, ...,
          2.41521866e-04,   9.00642152e-04,   5.82826854e-05],
       [ -5.14996032e-19,   1.64798730e-17,   1.62630326e-19, ...,
         -5.96311195e-19,   1.57353824e-03,  -8.13151629e-19],
       [  2.98155597e-19,  -2.16840434e-18,   0.00000000e+00, ...,
         -4.33680869e-19,  -8.67361738e-19,   1.35525272e-19],
       [  2.39945655e-04,   4.27605676e-03,   7.42743450e-04, ...,
          9.67587611e-04,   3.03043404e-04,   5.94420561e-04]])

In [33]:
import matplotlib.pyplot as plt

In [45]:
print(pd.crosstab(y_train, km.labels_,rownames = ["Actuall"],colnames = ["Predicted"]) )

Predicted      0    1   2    3
Actuall                       
RELIGION    1650   38  99  115
SCIENCE      801   56   0  759
STYLE       1341  235   0  127
WEIRD NEWS  1830   69   0  123


In [38]:
x_test_2

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

##### Books: 
##### STATISTICS_FOR_MACHINE_LEARNING<br>BUILDING MACHINE LEARNING SYSTEMS WITH PYTHON